In [34]:
import swisseph as swe
from datetime import datetime

In [35]:
#set sidereal mode (Lahiri)
swe.set_sid_mode(swe.SIDM_LAHIRI)


In [36]:
#Birth data
year, month, day = 1998, 7, 3
hour = 5 + 30/60 # 5L30 AM
lat, lon = 28.6139, 77.2090

In [37]:
#Convert to Julian Day
jd = swe.julday(year, month, day, hour)

In [38]:
#Get Ascendant and Houses
cusps, ascmc = swe.houses(jd, lat, lon)


In [39]:
print("Ascendant degree: ", ascmc[0])
print("House cusps: ", cusps)

Ascendant degree:  171.852453392009
House cusps:  (171.852453392009, 199.46160860447836, 229.85561976524593, 261.5030772026693, 293.05086908536384, 323.5125331322043, 351.852453392009, 19.46160860447833, 49.85561976524593, 81.50307720266932, 113.05086908536381, 143.51253313220428)


In [40]:
# Extracting the Ascendant (Lagna) sign
asc_degree = ascmc[0]
asc_sign = int(asc_degree // 30) + 1 #+1 to make 1 = Aries, 2 = Taurus, ..., 12 = Pisces

In [41]:
RASHIS = {
    1: "Aries", 2: "Taurus", 3: "Gemini", 4: "Cancer", 
    5: "Leo", 6: "Virgo", 7: "Libra", 8: "Scorpio",
    9: "Sagittarius", 10: "Capricorn", 11: "Aquarius", 
    12: "Pisces"
}

In [42]:
#derivind sign of each house
house_signs = {}
for i in range(0, 12):
    degree  = cusps[i]
    rashi = int(degree // 30 ) + 1
    house_signs[i+1] = rashi


In [43]:
print(house_signs)

{1: 6, 2: 7, 3: 8, 4: 9, 5: 10, 6: 11, 7: 12, 8: 1, 9: 2, 10: 3, 11: 4, 12: 5}


In [44]:
PLANETS = {
    0: "Sun", 1: "Moon", 2: "Mercury", 3: "Venus", 4: "Mars",
    5: "Jupiter", 6: "Saturn", 10: "Rahu", 11: "Ketu"
}

In [45]:
planet_positions = {}

In [46]:
for pid in PLANETS:
    if pid == 11: #ketu = 180 degree from Rahu
        rahu_data = swe.calc_ut(jd, 10)[0]
        rahu_lon = rahu_data[0]
        lon = (rahu_lon + 180) % 360
    else:
        data = swe.calc_ut(jd, pid)[0]
        lon = data[0]
    
    rashi = int(lon // 30) + 1
    planet_positions[PLANETS[pid]] = {
        'longitude': lon,
        'rashi': rashi
    }

planet_positions

{'Sun': {'longitude': 101.10766300659522, 'rashi': 4},
 'Moon': {'longitude': 206.9037496571462, 'rashi': 7},
 'Mercury': {'longitude': 123.5331725543876, 'rashi': 5},
 'Venus': {'longitude': 70.35215826805117, 'rashi': 3},
 'Mars': {'longitude': 87.86739275453522, 'rashi': 3},
 'Jupiter': {'longitude': 357.70399479541413, 'rashi': 12},
 'Saturn': {'longitude': 32.03831549329988, 'rashi': 2},
 'Rahu': {'longitude': 154.02252959476456, 'rashi': 6},
 'Ketu': {'longitude': 334.0225295947646, 'rashi': 12}}

# Map Planets to their houses

In [47]:
def get_house_for_planet(degree, cusps):
    for i in range(12):
        start = cusps[i]
        end = cusps[(i+1)%12]

        if start < end:
            if start <= degree < end:
                return i + 1
        else:
            if degree >= start or degree < end:
                return i + 1
    return None

In [48]:
# Adding house numer
for pname in planet_positions:
    degree = planet_positions[pname]['longitude']
    house = get_house_for_planet(degree, cusps)
    planet_positions[pname]['house'] = house


In [49]:
#adding house sign

for pname in planet_positions:
    hnum = planet_positions[pname]['house']
    planet_positions[pname]['house_rashi'] = house_signs[hnum]

In [50]:
planet_positions

{'Sun': {'longitude': 101.10766300659522,
  'rashi': 4,
  'house': 10,
  'house_rashi': 3},
 'Moon': {'longitude': 206.9037496571462,
  'rashi': 7,
  'house': 2,
  'house_rashi': 7},
 'Mercury': {'longitude': 123.5331725543876,
  'rashi': 5,
  'house': 11,
  'house_rashi': 4},
 'Venus': {'longitude': 70.35215826805117,
  'rashi': 3,
  'house': 9,
  'house_rashi': 2},
 'Mars': {'longitude': 87.86739275453522,
  'rashi': 3,
  'house': 10,
  'house_rashi': 3},
 'Jupiter': {'longitude': 357.70399479541413,
  'rashi': 12,
  'house': 7,
  'house_rashi': 12},
 'Saturn': {'longitude': 32.03831549329988,
  'rashi': 2,
  'house': 8,
  'house_rashi': 1},
 'Rahu': {'longitude': 154.02252959476456,
  'rashi': 6,
  'house': 12,
  'house_rashi': 5},
 'Ketu': {'longitude': 334.0225295947646,
  'rashi': 12,
  'house': 6,
  'house_rashi': 11}}